In [10]:
# !pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

In [7]:
from hdfs import InsecureClient # 하둡 서버와 연결하는 client 객체 모듈(hdfs 접근)
import requests # web api 사용 위함
import json # json 형식으로 바꿔서 저장하기 위해 사용
import datetime as dt

### DataW에 DataL 의 필요 data 를 이동 저장 단계
- Lake 에서 HDFS로 이동 저장
1. 코로나 발생현황(일간데이터) : api 수집
2. 코로나 예방접종현황(일간데이터) : web 페이지 크롤링
3. 인구 사회 데이터 : 파일 데이터
- 1, 2번 데이터는 json 형식으로 변환 후 hdfs에 저장
- 3번 데이터는 파일 그대로 저장

In [12]:
## hdfs client 연결객체 생성
client = InsecureClient('http://localhost:9870', user = 'root')

## hdfs로 부터 읽기

In [13]:
# client 객체 사용 file 읽어오기
with client.read('/rdd/score.txt') as reader :
    score = reader.read() # byte code 로 읽어옴

In [14]:
score

b'\xea\xb9\x80\xec\xb2\xa0\xec\x88\x98 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 50\r\n\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 80\r\n\xec\x9e\x84\xea\xba\xbd\xec\xa0\x95 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 60\r\n\xec\x9e\x84\xec\x9a\x94\xed\x99\x98 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 100\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xec\x9d\xb4\xec\x9c\xa4\xec\x97\xb4 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 90\r\n'

In [9]:
# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90



## hdfs에 쓰기

In [23]:
# hdd(note 디렉터리)에 있는 data 를 hdfs 로 이동
# 1. 로컬 데이터 파일을 열기
# 2. hdfs client 통해서 쓰기 모듈 사용
# 3. 로컬 파일의 내용을 한줄씩 읽어서 -> hdfs에 쓰기
with open('./data/corona_data/sido_area.csv',encoding='CP949') as reader, client.write('./data/corona_data/sido_area_tmp.csv') as writer:
    for line in reader :
        writer.write(line.encode('CP949'))

ERROR:hdfs.util:Exception in child.
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/util.py", line 76, in consumer
    self._consumer(data)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/client.py", line 515, in consumer
    raise _to_error(res)
hdfs.util.HdfsError: Permission denied: user=root, access=WRITE, inode="/user":lab09:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:242)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess

HdfsError: Permission denied: user=root, access=WRITE, inode="/user":lab09:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:242)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1886)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.resolvePathForStartFile(FSDirWriteFileOp.java:323)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2685)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2625)
	at or

In [21]:
# 폴더 전송
# aws 서버에서 진행하고 있기 때문에 권한이 없으면 오류가 발생한다. 밑에서도 계속 발생할 예정임
client.upload('/corona_data/loc', './data/corona_data')

ERROR:hdfs.client:Error while uploading. Attempting cleanup.
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/client.py", line 643, in upload
    _upload(path_tuple)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/client.py", line 574, in _upload
    self.write(_temp_path, wrap(reader, chunk_size, progress), **kwargs)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/client.py", line 520, in write
    consumer(data)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/hdfs/client.py", line 515, in consumer
    raise _to_error(res)
hdfs.util.HdfsError: Permission denied: user=root, access=WRITE, inode="/corona_data":lab09:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermission

HdfsError: Permission denied: user=root, access=WRITE, inode="/corona_data":lab09:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:242)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1886)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.resolvePathForStartFile(FSDirWriteFileOp.java:323)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2685)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2625)

## hdfs에 수정하기
- hdfs.client.write (파일명, append = True)

In [26]:
client.write('/rdd/score.txt', '최연성 장고 100'.encode('UTF-8'), append = True)

HdfsError: Permission denied: user=root, access=WRITE, inode="/rdd/score.txt":lab09:supergroup:-rw-r--r--
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:352)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermissionWithContext(FSPermissionChecker.java:370)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:240)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPathAccess(FSDirectory.java:1877)
	at org.apache.hadoop.hdfs.server.namenode.FSDirAppendOp.appendFile(FSDirAppendOp.java:101)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.appendFile(FSNamesystem.jav

## hdfs 권한 수정

In [29]:
client.set_permission('/corona_data/', '777')

HdfsError: Permission denied. user=root is not the owner of inode=/corona_data

## hdfs 삭제

In [19]:
client.delete('/corona_data/loc/sidd_area.xlsx')

False

### 공공데이터 REST_API 통해서 데이터 수집 후 hdfs에 저장

1. 일간 코로나 발생 데이터(전국)

## REST_API로 데이터를 호출해 HDFS에 저장

In [4]:
def executeRestApi(method, url, headers, params):  
    
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get":
        res = requests.get(url, params=params, headers=headers)
    else:
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text

### 기준일자 함수

In [5]:
def cal_std_day(befor_day):
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [6]:
cal_std_day(1)

NameError: name 'dt' is not defined

### logger
- log 를 기록해주는 객체(관련 패키지 존재 : logging)
- 외부 연결 시 오류 상태 확인 위해서 사용자 정의 log 사용 가능해야 한다.
- restapi 는 외부 서버 연결이므로 log 기록
- 로그 관련 디렉터리 
    1. 로그 관련 패키지 import(import logging)
    2. 로그 관리 객체 logger 생성(logging.getLogger('로거이름'))
    3. 로그 파일 핸들러 생성(logging.FileHandler('로그파일명.log))
    4. 로거에 파일핸들러 추가

In [36]:
import logging

In [38]:
co_logger = logging.getLogger('conrona_api')
f_handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'log')
co_logger.addHandler(f_handler)

In [39]:
# 이렇게 문자열로 찍을 수도 있음
# json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.
co_logger.error('corona_patient_'+cal_std_day(0)+'.json 다운로드 실패')

ERROR:conrona_api:corona_patient_2023-12-22.json 다운로드 실패


- 위에서 강제 발생시킨 에러에 대한 log 는 ./log/rest_api/'+cal_std_day(0)+'log 로그 파일에서 확인

### api 호출

In [48]:
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
service_key = 'vpA5w%2Frqnqv5OVUvdS1jzdjrdM3lN%2B3Omjbnx64dXdMBMpL7xbLreYIFaigNiexQelZHSsPNMGL3A91tHKF1YQ%3D%3D'
file_dir = '/corona_data/patient/'

In [49]:
# std_day : 오늘부터 얼머나 이전 데이터인지 나타냄 0 : 당일/ 1: 어제 / 365 : 1년전
def create_param(std_day):
    return {'serviceKey' : service_key,
           'pageNo' : 1,
           'numOfRows' : 500,
           'apiType' : 'JSON',
            'std_day': cal_std_day(std_day)}

In [50]:
create_param(365)

{'serviceKey': 'vpA5w%2Frqnqv5OVUvdS1jzdjrdM3lN%2B3Omjbnx64dXdMBMpL7xbLreYIFaigNiexQelZHSsPNMGL3A91tHKF1YQ%3D%3D',
 'pageNo': 1,
 'numOfRows': 500,
 'apiType': 'JSON',
 'std_day': '2022-12-22'}

In [53]:
for i in range(365, 367) :
    params = create_param(i) # api 요청 파라미터 생성
    log_dict = {
        'is_sucess' : 'Fail'
        ,'type' : 'corona_patient'
        ,'std_day' : params['std_day']
        ,'params' : params
    }
    try:
        res = executeRestApi('get',url, {}, params) # 공공데이터 api에서 추출
        file_name = 'corona_patient_'+cal_std_day(i)+'.json' # 저장 파일명 생성
        client.write(file_dir+file_name,res, encoding = 'utf=8') # hdfs 에 저장
    
    except Exception as e :
        log_dict['err_msg'] = e.__str__() # exception 객체 내 에러메시지를 반환
        log_json = json.dumps(log_dict, ensure_ascii=False) # json형태로 기록 
        co_logger.error(log_json) # 로그파일에 기록
        

ERROR:conrona_api:{"is_sucess": "Fail", "type": "corona_patient", "std_day": "2022-12-22", "params": {"serviceKey": "vpA5w%2Frqnqv5OVUvdS1jzdjrdM3lN%2B3Omjbnx64dXdMBMpL7xbLreYIFaigNiexQelZHSsPNMGL3A91tHKF1YQ%3D%3D", "pageNo": 1, "numOfRows": 500, "apiType": "JSON", "std_day": "2022-12-22"}, "err_msg": "Permission denied: user=root, access=WRITE, inode=\"/corona_data\":lab09:supergroup:drwxr-xr-x\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:242)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirecto

## 웹크롤링 hdfs 저장

- 예방접종현황(백신 접종현황)

In [35]:
# pip install BeautifulSoup4
from bs4 import BeautifulSoup

url = 'https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000'

In [36]:
response = executeRestApi('get', url, {},{})
# response

In [37]:
soup = BeautifulSoup(response, 'html.parser')

In [38]:
#content > div.data_table.tbl_scrl_mini > table > tbody > tr

In [39]:
trs = soup.select('#content > div.data_table.tbl_scrl_mini > table > tbody > tr')

In [40]:
len(trs)

18

In [41]:
col = ['loc', 'new', 'accumulate', 'rate']
data = []

In [44]:
for idx, tr in enumerate(trs):
    if idx == 0:
        continue # idx 0번 데이터는 전국 데이터 제외해야 하므로 continue
    tuple_t = []
    # 지역 값
    loc_th = tr.find('th')
    # 접종관련 데이터들
    tds = tr.select('td')
    # 지역 이름을 추가
    tuple_t.append(loc_th.text)
    # 접종관련 데이터 추가
    tuple_t.append(tds[0].text.replace(' ','').replace('\r\n', '').replace(',',''))
    tuple_t.append(tds[1].text.replace(' ','').replace('\r\n', '').replace(',',''))
    tuple_t.append(tds[2].text.replace(' ','').replace('\r\n', '').replace(',',''))
    data.append(dict(zip(cols, tuple_t)))

In [45]:
# json 에 포함시킬 meta data 구성
res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
                'loc':'지역'
                ,'new':'신규접종'
                ,'accumulate':'누적접종' 
               ,'rate' : '접종률'
                },
        'stdDay' : cal_std_day(0)
    },
    'data' : data
}
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'new': '신규접종', 'accumulate': '누적접종', 'rate': '접종률'},
  'stdDay': '2023-12-22'},
 'data': [{'loc': '서울', 'v1': '1289', 'v2': '686841', 'v3': '39.6%'},
  {'loc': '부산', 'v1': '396', 'v2': '272263', 'v3': '36.3%'},
  {'loc': '대구', 'v1': '305', 'v2': '150341', 'v3': '32%'},
  {'loc': '인천', 'v1': '445', 'v2': '198337', 'v3': '39.8%'},
  {'loc': '광주', 'v1': '124', 'v2': '109590', 'v3': '46.5%'},
  {'loc': '대전', 'v1': '141', 'v2': '103662', 'v3': '42.1%'},
  {'loc': '울산', 'v1': '179', 'v2': '65936', 'v3': '37.3%'},
  {'loc': '세종', 'v1': '43', 'v2': '18264', 'v3': '42.7%'},
  {'loc': '경기', 'v1': '1159', 'v2': '861348', 'v3': '40.5%'},
  {'loc': '강원', 'v1': '349', 'v2': '141478', 'v3': '38.5%'},
  {'loc': '충북', 'v1': '318', 'v2': '142307', 'v3': '42.5%'},
  {'loc': '충남', 'v1': '183', 'v2': '189657', 'v3': '41.4%'},
  {'loc': '전북', 'v1': '245', 'v2': '196850', 'v3': '46.2%'},
  {'loc': '전남', 'v1': '272', 'v2': '228066', 'v3': '48%'},

In [47]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(1)+'.json'
client.write(file_dir+file_name, json.dumps(res, ensure_ascii = False), encoding='utf=8')

NameError: name 'client' is not defined

In [48]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(1)+'.json'
vaccine = spark.read.json(file_dir+file_name)
vaccine.show()

AnalysisException: Path does not exist: hdfs://localhost:9000/corona_data/vaccine/corona_vaccine_2023-12-21.json

### 위 크롤링된 데이터가 전 연령에 대한 백신 관련 데이터가 아니므로 다른 파일 사용함

In [22]:
import pandas as pd

In [24]:
fin_vac = pd.read_csv('./data/corona_data/코로나백신접종완료_2022_12.csv', index_col = 0)

In [25]:
fin_vac.head()

,LOC,V1,V2,V3
0,제주,591058,585437.0,439106.0
1,서울,8335298,8257768.0,6076429.0
2,부산,2879900,2851362.0,2140773.0
3,대구,2018604,1995783.0,1405163.0
4,광주,1261981,1250568.0,968947.0


In [30]:
tmp = fin_vac.loc[4]
tmp.LOC

'광주'

In [27]:
cols = ['loc', 'v1', 'v2', 'v3']
data = []

In [32]:
for idx in fin_vac.index:
    tuple_t = []
    tmp = fin_vac.loc[idx]
    tuple_t.append(str(tmp.LOC))
    tuple_t.append(str(tmp.V1))
    tuple_t.append(str(tmp.V2))
    tuple_t.append(str(tmp.V3))
    data.append(dict(zip(cols, tuple_t)))

In [33]:
data

[{'loc': '제주', 'v1': '591058', 'v2': '585437.0', 'v3': '439106.0'},
 {'loc': '서울', 'v1': '8335298', 'v2': '8257768.0', 'v3': '6076429.0'},
 {'loc': '부산', 'v1': '2879900', 'v2': '2851362.0', 'v3': '2140773.0'},
 {'loc': '대구', 'v1': '2018604', 'v2': '1995783.0', 'v3': '1405163.0'},
 {'loc': '광주', 'v1': '1261981', 'v2': '1250568.0', 'v3': '968947.0'},
 {'loc': '인천', 'v1': '2574233', 'v2': '2550081.0', 'v3': '1915266.0'},
 {'loc': '대전', 'v1': '1245473', 'v2': '1233376.0', 'v3': '909102.0'},
 {'loc': '경기', 'v1': '11837174', 'v2': '11723258.0', 'v3': '8707827.0'},
 {'loc': '세종', 'v1': '300568', 'v2': '296859.0', 'v3': '211095.0'},
 {'loc': '경남', 'v1': '2882823', 'v2': '2853755.0', 'v3': '2145652.0'},
 {'loc': '경북', 'v1': '2293225', 'v2': '2269593.0', 'v3': '1727188.0'},
 {'loc': '전남', 'v1': '1655633', 'v2': '1641348.0', 'v3': '1365245.0'},
 {'loc': '전북', 'v1': '1593950', 'v2': '1581091.0', 'v3': '1285767.0'},
 {'loc': '충남', 'v1': '1902008', 'v2': '1884584.0', 'v3': '1479736.0'},
 {'loc': '충북

In [46]:
# json 에 포함시킬 meta data 구성
res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
                'loc':'지역'
                ,'V1':'1차 접종 누적'
                ,'V2':'2차 접종 누적' 
                ,'V3':'3차 접종 누적'
                },
        'stdDay' : cal_std_day(365)
    },
    'data' : data
}
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'V1': '1차 접종 누적', 'V2': '2차 접종 누적', 'V3': '3차 접종 누적'},
  'stdDay': '2022-12-22'},
 'data': [{'loc': '서울', 'v1': '1289', 'v2': '686841', 'v3': '39.6%'},
  {'loc': '부산', 'v1': '396', 'v2': '272263', 'v3': '36.3%'},
  {'loc': '대구', 'v1': '305', 'v2': '150341', 'v3': '32%'},
  {'loc': '인천', 'v1': '445', 'v2': '198337', 'v3': '39.8%'},
  {'loc': '광주', 'v1': '124', 'v2': '109590', 'v3': '46.5%'},
  {'loc': '대전', 'v1': '141', 'v2': '103662', 'v3': '42.1%'},
  {'loc': '울산', 'v1': '179', 'v2': '65936', 'v3': '37.3%'},
  {'loc': '세종', 'v1': '43', 'v2': '18264', 'v3': '42.7%'},
  {'loc': '경기', 'v1': '1159', 'v2': '861348', 'v3': '40.5%'},
  {'loc': '강원', 'v1': '349', 'v2': '141478', 'v3': '38.5%'},
  {'loc': '충북', 'v1': '318', 'v2': '142307', 'v3': '42.5%'},
  {'loc': '충남', 'v1': '183', 'v2': '189657', 'v3': '41.4%'},
  {'loc': '전북', 'v1': '245', 'v2': '196850', 'v3': '46.2%'},
  {'loc': '전남', 'v1': '272', 'v2': '228066', 'v3': '48%'

In [ ]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365)+'.json'
client.write(file_dir+file_name, json.dumps(res, ensure_ascii = False), encoding='utf=8')

In [ ]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365)+'.json'
vaccine = spark.read.json(file_dir+file_name)
vaccine.show()